Experiment :- WAP to train and evaluate a convolutional neural network using Keras Library to
classify MNIST fashion dataset. Demonstrate the effect of filter size, regularization,
batch size and optimization algorithm on model performance

Model Architecture:
1.Input Layer:

• The model accepts input images of size 28x28x1, where 28x28 is the image dimension and 1 is the number of channels (grayscale).

2.First Convolutional Layer:

• Conv2D: The first convolutional layer has 32 filters, each of size (3x3), with the activation function set to ReLU.

• Input Shape: The input shape for this layer is (28, 28, 1), meaning the image is 28x28 pixels with 1 color channel (grayscale).

• Padding: It uses valid padding, which means no padding is applied to the image, so the spatial dimensions will shrink after the convolution operation.

• After applying this convolution, the output shape will be (26, 26, 32) because a 3x3 filter reduces the image size by 2 in each dimension.

3.First MaxPooling Layer:

• MaxPooling2D: This layer performs a 2x2 max pooling operation, which downsamples the feature map by taking the maximum value in each 2x2 patch.

• After pooling, the spatial dimensions are halved, so the output shape is reduced to (13, 13, 32).

4.Second Convolutional Layer:

• Conv2D: The second convolutional layer has 64 filters, each of size (3x3), with ReLU activation.

• This layer takes the output from the previous max-pooling layer, which has the shape (13, 13, 32).

• After applying the convolution, the output shape becomes (11, 11, 64).

5.Second MaxPooling Layer:

• MaxPooling2D: Another 2x2 max pooling operation, which reduces the spatial dimensions by half again. The output shape is now (5, 5, 64).

6.Third Convolutional Layer:

• Conv2D: The third convolutional layer has 128 filters, each of size (3x3), with ReLU activation.

• The input to this layer has the shape (5, 5, 64).

• After applying the convolution, the output shape becomes (3, 3, 128).

7.Third MaxPooling Layer:

• MaxPooling2D: A final 2x2 max pooling operation. This reduces the spatial dimensions once more, and the output shape is now (1, 1, 128).

8.Flatten Layer:

• Flatten: This layer flattens the 3D feature map (1, 1, 128) into a 1D vector of size 128. Flattening is necessary before feeding the data into fully connected layers (Dense layers).

9.Fully Connected Layer (Dense):

• Dense: The first dense layer has 128 units and uses ReLU activation.

This layer processes the flattened feature map and outputs a vector of size 128.

10.Output Layer:

• Dense: The output layer has 10 units (one for each class in the Fashion MNIST dataset) with a softmax activation function. This outputs a probability distribution over the 10 classes, and the class with the highest probability is chosen as the predicted label

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers

# Step 1: Load and preprocess the dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape input to match CNN input format (28x28x1)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Step 2: Build a CNN model
def build_cnn_model(filter_size=(3,3), regularizer=None, batch_size=64, optimizer='adam'):
    model = models.Sequential()

    # Add convolutional layer
    model.add(layers.Conv2D(32, filter_size, activation='relu', input_shape=(28, 28, 1), kernel_regularizer=regularizer))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))  # Pooling reduces spatial size
    model.add(layers.Conv2D(64, filter_size, activation='relu', kernel_regularizer=regularizer))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))  # Pooling reduces spatial size
    model.add(layers.Conv2D(128, filter_size, activation='relu', kernel_regularizer=regularizer))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))  # Pooling reduces spatial size

    # Flatten and dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    # Compile the model with the given optimizer
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Step 3: Train and evaluate the model for different configurations

# Helper function to train and evaluate
def train_evaluate_model(filter_size, regularizer, batch_size, optimizer):
    model = build_cnn_model(filter_size, regularizer, batch_size, optimizer)
    
    history = model.fit(x_train, y_train, epochs=5, batch_size=batch_size, validation_data=(x_test, y_test), verbose=1)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
    return test_acc

# Experiment with different configurations
configurations = [
    {'filter_size': (3, 3), 'regularizer': None, 'batch_size': 64, 'optimizer': 'adam'},
    {'filter_size': (3, 3), 'regularizer': None, 'batch_size': 64, 'optimizer': 'adam'},
    {'filter_size': (3, 3), 'regularizer': regularizers.l2(0.01), 'batch_size': 64, 'optimizer': 'adam'},
    {'filter_size': (3, 3), 'regularizer': None, 'batch_size': 32, 'optimizer': 'adam'},
    {'filter_size': (3, 3), 'regularizer': None, 'batch_size': 64, 'optimizer': 'sgd'},
]

# Train and evaluate each configuration
results = []

for config in configurations:
    acc = train_evaluate_model(config['filter_size'], config['regularizer'], config['batch_size'], config['optimizer'])
    results.append((config, acc))

# Print the results
for config, acc in results:
    print(f"Config: {config}, Test Accuracy: {acc:.4f}")


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 56s 55ms/step - accuracy: 0.6876 - loss: 0.8506 - val_accuracy: 0.8134 - val_loss: 0.5050
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 80s 53ms/step - accuracy: 0.8448 - loss: 0.4276 - val_accuracy: 0.8587 - val_loss: 0.3975
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 49s 53ms/step - accuracy: 0.8677 - loss: 0.3560 - val_accuracy: 0.8659 - val_loss: 0.3652
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 50s 53ms/step - accuracy: 0.8836 - loss: 0.3157 - val_accuracy: 0.8770 - val_loss: 0.3416
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 52s 55ms/step - accuracy: 0.8929 - loss: 0.2891 - val_accuracy: 0.8822 - val_loss: 0.3218
313/313 - 3s - 8ms/step - accuracy: 0.8822 - loss: 0.3218
Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 57s 56ms/step - accuracy: 0.6956 - loss: 0.8356 - val_accuracy: 0.8343 - val_loss: 0.4462
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 45ms/step - accuracy: 0.8502 - loss: 0.4117 - val_accuracy: 0.8545 - val_loss: 0.3957
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 

• Result:- validation accuracy increases over epochs, but it fluctuates compared to training accuracy. The model is generalizing well, with a slight increase in validation accuracy towards the end.

If the validation accuracy were to drop drastically while the training accuracy kept increasing, it would indicate overfitting, but in this case, they track quite similarly, which is good.